In [ ]:
# Aadil Nazir UE208001 UIET PU Chandigarh...
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import logging
from torchvision.models import ResNet18_Weights

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Fix 
torch.manual_seed(42)

# Hyperparameters
batch_size = 64  # Increased batch size 
learning_rate = 0.001  
num_epochs = 10  # Increased

# Data preparation with additional augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Added color jitter
    transforms.RandomAffine(10, shear=10),  # Added random affine for more augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization for CIFAR-10
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Device setup (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Load a pre-trained ResNet18 model with the correct weight enum
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # or ResNet18_Weights.DEFAULT
model.fc = nn.Linear(model.fc.in_features, 10)  
model = model.to(device)  


for param in model.parameters():
    param.requires_grad = True  # Unfreeze all layers


optimizer = optim.Adam(model.parameters(), lr=learning_rate)


criterion = nn.CrossEntropyLoss()


scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce lr every 5 epochs

# Training function
def train_finetune(model, train_loader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 100 == 0:
                avg_loss = running_loss / 100
                logging.info(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i}], Loss: {avg_loss:.4f}")
                running_loss = 0.0
        
        # Step the learning rate scheduler
        scheduler.step()

        # Evaluate model after each epoch
        logging.info(f"Evaluating the model after epoch {epoch+1}...")
        test(model, test_loader)

# Testing function
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    logging.info(f"Test Accuracy: {accuracy:.2f}%")

# Fine-tune the pre-trained model
logging.info("Fine-tuning the pre-trained model...")
train_finetune(model, train_loader, criterion, optimizer, scheduler, num_epochs)

# Test the fine-tuned model
logging.info("Evaluating the fine-tuned model...")
test(model, test_loader)

# Train the model from scratch for comparison
logging.info("Training a new model from scratch...")
model_scratch = models.resnet18(weights=None) 
model_scratch.fc = nn.Linear(model_scratch.fc.in_features, 10)
model_scratch = model_scratch.to(device)

optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=learning_rate)

scheduler_scratch = optim.lr_scheduler.StepLR(optimizer_scratch, step_size=5, gamma=0.1)
# fine_tune model
train_finetune(model_scratch, train_loader, criterion, optimizer_scratch, scheduler_scratch, num_epochs)

# Test the scratch model
logging.info("Evaluating the Model Trained from Scratch...")
test(model_scratch, test_loader)

Files already downloaded and verified
Files already downloaded and verified


2024-12-14 14:01:38,365 - INFO - Using device: cpu
2024-12-14 14:01:40,430 - INFO - Fine-tuning the pre-trained model...
2024-12-14 14:04:28,374 - INFO - Epoch [1/10], Batch [100], Loss: 1.8689
2024-12-14 14:07:56,087 - INFO - Epoch [1/10], Batch [200], Loss: 1.5549
2024-12-14 14:11:50,847 - INFO - Epoch [1/10], Batch [300], Loss: 1.4071
2024-12-14 14:15:29,767 - INFO - Epoch [1/10], Batch [400], Loss: 1.3384
2024-12-14 14:19:06,819 - INFO - Epoch [1/10], Batch [500], Loss: 1.3049
2024-12-14 14:22:53,448 - INFO - Epoch [1/10], Batch [600], Loss: 1.2098
2024-12-14 14:26:36,442 - INFO - Epoch [1/10], Batch [700], Loss: 1.1723
2024-12-14 14:29:42,278 - INFO - Evaluating the model after epoch 1...
2024-12-14 14:30:29,713 - INFO - Test Accuracy: 54.01%
2024-12-14 14:34:32,720 - INFO - Epoch [2/10], Batch [100], Loss: 2.3324
2024-12-14 14:38:07,707 - INFO - Epoch [2/10], Batch [200], Loss: 2.0086
2024-12-14 14:41:50,206 - INFO - Epoch [2/10], Batch [300], Loss: 1.8264
